In [1]:
import pandas as pd
import numpy as np

import yfinance as yf

/Users/markhendricks/Projects/mlp-portfolio-mgmt-2025/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
TICKS =  ['SPY','AAPL','NVDA','MSFT','GOOGL','AMZN','META','TSLA','AVGO','BRK-B','LLY']

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [3]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]

In [4]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,46716826,623795240960,The trust seeks to achieve its investment obje...
AAPL,Apple Inc.,EQUITY,USD,36473385,NaN,"Apple Inc. designs, manufactures, and markets ..."
NVDA,NVIDIA Corporation,EQUITY,USD,218985854,NaN,NVIDIA Corporation provides graphics and compu...
MSFT,Microsoft Corporation,EQUITY,USD,14863983,NaN,Microsoft Corporation develops and supports so...
GOOGL,Alphabet Inc.,EQUITY,USD,24249066,NaN,Alphabet Inc. offers various products and plat...
AMZN,"Amazon.com, Inc.",EQUITY,USD,23698805,NaN,"Amazon.com, Inc. engages in the retail sale of..."
META,"Meta Platforms, Inc.",EQUITY,USD,9594911,NaN,"Meta Platforms, Inc. engages in the developmen..."
TSLA,"Tesla, Inc.",EQUITY,USD,71603819,NaN,"Tesla, Inc. designs, develops, manufactures, l..."
AVGO,Broadcom Inc.,EQUITY,USD,18145972,NaN,"Broadcom Inc. designs, develops, and supplies ..."


In [5]:
STARTDATE = '2015-01-01'
ENDDATE = '2024-12-31'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE, auto_adjust=False)['Adj Close']

[*********************100%***********************]  11 of 11 completed


In [6]:
AGG = 'W-FRI'
prices = df.resample(AGG).last().iloc[:-1,:]
prices.index.name = 'date'

# eliminate timezones
prices.index = prices.index.tz_localize(None)


rets = prices.pct_change().dropna()

In [7]:
rets

Ticker,AAPL,AMZN,AVGO,BRK-B,GOOGL,LLY,META,MSFT,NVDA,SPY,TSLA
date,,,,,,,,,,,
2015-01-09,0.024513,-0.037566,0.048056,0.002011,-0.054442,-0.001855,-0.009050,0.009196,-0.009438,-0.005744,-0.057681
2015-01-16,-0.053745,-0.020847,-0.010295,-0.001739,0.019452,0.010725,-0.032930,-0.020132,0.001003,-0.012827,-0.065760
2015-01-23,0.065950,0.074465,0.030437,-0.000603,0.061689,0.020515,0.035249,0.020329,0.037575,0.016565,0.042575
2015-01-30,0.036998,0.134895,-0.038325,-0.034938,-0.008119,-0.001802,-0.024669,-0.143705,-0.072912,-0.026931,0.011476
2015-02-06,0.019114,0.055708,0.018079,0.043569,-0.006827,-0.022778,-0.018970,0.049752,0.062500,0.030584,0.067584
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-29,0.032453,0.054637,-0.013091,0.013534,0.025431,0.063288,0.027149,0.015492,-0.026065,0.011822,-0.020989
2024-12-06,0.023217,0.092068,0.107663,-0.025920,0.034093,0.039429,0.086102,0.047490,0.030378,0.008730,0.127651
2024-12-13,0.021784,0.001894,0.252158,-0.026780,0.087731,-0.045469,-0.005483,0.008341,-0.057498,-0.005923,0.120780


In [8]:
with pd.ExcelWriter('../data/exam_1_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'info')
    prices.to_excel(writer, sheet_name= 'prices')
    rets.to_excel(writer, sheet_name='total returns')